# Building The Design Space for Lyophilization-Edge and Core

In [1]:
#Outputs:
    #Drying Time
    #Product Temperature

In [1]:
#Libraries
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import least_squares
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')
import math
from mpl_toolkits import mplot3d

### Model Constant Inputs

In [6]:
#Input Parameters
SAin = float(input("Enter Surface Area In(cm^2): ")) #5.982843996
print(SAin)
vial_capac = float(input("Enter Vial Capac: ")) #20
print(vial_capac)
vial_numbers = float(input("Enter Vial Numbers(filled): ")) #86
print(vial_numbers)
ice_den = float(input("Enter Ice Density: ")) #0.918
print(ice_den)
den = float(input("Enter Density: ")) #1.036
print(den)
fill_volume = float(input("Enter Fill Volume(ml): ")) #6.3
print(fill_volume)
water_cont = float(input("Enter Water Content: ")) #0.484
print(water_cont)
Din = float(input("Enter Din: ")) #2.76
print(Din)
Dout = float(input("Enter Dout: ")) #3
print(Dout)
dry_cake=(fill_volume*den)/(SAin*ice_den)
print("Dry Cake Initial Height:",dry_cake)
lam=0.00358

Enter Surface Area In(cm^2): 5.982843996
5.982843996
Enter Vial Capac: 20
20.0
Enter Vial Numbers(filled): 86
86.0
Enter Ice Density: 0.918
0.918
Enter Density: 1.036
1.036
Enter Fill Volume(ml): 6.3
6.3
Enter Water Content: 0.484
0.484
Enter Din: 2.76
2.76
Enter Dout: 3
3.0
Dry Cake Initial Height: 1.1883652534350033
Dry Cake Initial Height: 1.1883652534350033


##  Iterative Code To Create Design Space

In [3]:
#Set Sampling Rate for Chamber Pressure and Shelf Temperatures(inputs to our model)
sr_t = float(input("Enter Temperature Sampling Rate: ")) #5
print(sr_t)
sr_p = float(input("Enter Pressure Sampling Rate: ")) #5
print(sr_p)
T_max = float(input("Enter Maximum Temperature(C): ")) #10
print(T_max)
T_min = float(input("Enter Minimum Temperature(C): ")) #-30
print(T_min)
P_max = float(input("Enter Maximum Pressure(C): ")) #250
print(P_max)
P_min = float(input("Enter Minimum Pressure(C): ")) #50
print(P_min)

temp=np.arange(T_min,T_max,sr_t) #Sample temperature 
press=np.arange(P_min,P_max,sr_p) #Sample pressure 

Enter Temperature Sampling Rate: 5
5.0
Enter Pressure Sampling Rate: 5
5.0
Enter Maximum Temperature(C): 10
10.0
Enter Minimum Temperature(C): -30
-30.0
Enter Maximum Pressure(C): 250
250.0
Enter Minimum Pressure(C): 50
50.0


#Note: 
Using 2NN vial for the edge case. Took average for resistance parameters from F4 HH and F4 LL cycles. 

## Cake Resistance Parameters

In [4]:
#Set Cake Resistance Parameters
a = float(input("Enter your A parameter for resistance: ")) #-8.85164801e-02
print(a)
b = float(input("Enter your B parameter for resistance: ")) #9.04999955e+04
print(b)
c = float(input("Enter your C parameter for resistance: ")) #6.50812284e+00
print(c)

cake_res_param=np.zeros(3)
cake_res_param[0]=a
cake_res_param[1]=b
cake_res_param[2]=c

Enter your A parameter for resistance: -8.85164801e-02
-0.0885164801
Enter your B parameter for resistance: 9.04999955e+04
90499.9955
Enter your C parameter for resistance: 6.50812284e+00
6.50812284


## Start Iterations

In [12]:
#Create Matrices to Store Predicted Drying Time and Product Temp at Each Pressure/Temp combo
dt=np.zeros((len(temp),len(press)))
tp=np.zeros((len(temp),len(press)))
p_min=np.zeros(((len(temp),len(press),59)))
p_min_param=np.array([5.12,18.829,0])
#Start Iterations
for i in range(len(temp)):
    for j in range(len(press)):
        print("Current Pressure/Temp Combo:",temp[i],"C",press[j],"mTorr")
        
        #Set initial value for percent removed water
        prw_v=0
        #Set counter variable
        count=10
        
        while prw_v<100 and count<60: # keep iterating until prw reaches closest value to 100
            print("Current Iteration:", count-9)
            #Set Initial Arrays that will keep changing in length
            cycle_time=np.arange(18,count+18,1)
            Kv_values=np.repeat(0.00020961432864185497,count) #From Kv Fitting model, doesn't change based on vial configuration
            shelf_temp=np.repeat(temp[i],count) #In degrees Celsius
            cham_pres=np.repeat(press[j]/1000,count) #In Torr

            #OLS Function
            def ols_fit():
                #Define Parameter Set(Initial)
                #T_p_0=((df2[['TC AVG']]).values)[16:count+16] #Used thermocouple average as a starting point for the product temperature fitting
                T_p_0=np.repeat(-45,count)
                #Define Residuals Function
                def residuals(T_p): #T_p is the parameter we are fitting
                    #Need to calculate initial condition for cake resistance and Tice based on initial T_p_0 guess
                    #Go through all steps here
                    #Define function to calculate cake resistance and Tice
                    def intermed(T_p):
                        hf=(shelf_temp-T_p)*Kv_values
                        sh=np.zeros(hf.size)
                        for i in range(len(hf)):
                            if i==0:
                                sh[i]=0
                            else:
                                sh[i]=sh[i-1]+(hf[i]+hf[i-1])*(cycle_time[i]-cycle_time[i-1])*3600/2*Dout*Dout*np.pi/4
                        msi=sh/676
                        pdl=msi/(water_cont*ice_den*Din*Din*np.pi/4)
                        cake_res=(cake_res_param[0]+((cake_res_param[1]*pdl)/(1+cake_res_param[2]*pdl)))/1000 #Use 2NN case for edge
                        Tice_surf=T_p-hf*(dry_cake-pdl)/lam
                        return cake_res, Tice_surf
                    outputs=intermed(T_p) #variable to store cake res and Tice_surf
                    p_pred=np.exp(24.01849-6144.96/(outputs[1].astype("float64")+273.0)) #Have to set initial condition for Tice_surf
                    #p_pred=np.exp(24.01849-6144.96/(outputs[0]+273.0)) #Have to set initial condition for Tice_surf
                    p_real=cham_pres+outputs[0]*3600*((Dout*Dout)/(Din*Din))*Kv_values*(shelf_temp-T_p)/676 #Have to set initial condition for cake_res
                    p_real=np.array(p_real, dtype=float)
                    res=p_pred-p_real
                    return res
                #Least Squares Fitting
                lsq=least_squares(residuals,T_p_0, loss='soft_l1', f_scale=0.1)

                #Fitted Product Temperature
                T_p_fit=lsq.x

                return T_p_fit #final outcome T_p 
            ####   At this point you have fitted product temperature ####
            T_product=ols_fit()

            #Function to calculate all intermediate values used to fit the T_product
            def param(T_p):
                hf=(shelf_temp-T_p)*Kv_values
                sh=np.zeros(hf.size)
                for i in range(len(hf)):
                    if i==0:
                        sh[i]=0
                    else:
                        sh[i]=sh[i-1]+(hf[i]+hf[i-1])*(cycle_time[i]-cycle_time[i-1])*3600/2*Dout*Dout*np.pi/4
                msi=sh/676
                pdl=msi/(water_cont*ice_den*Din*Din*np.pi/4)
                cake_res=cake_res_param[0]+((cake_res_param[1]*pdl)/(1+cake_res_param[2]*pdl)) 
                Tice_surf=T_p-hf*(dry_cake-pdl)/lam
                return cake_res, Tice_surf, hf, msi, sh, pdl

            #Find all intermediate values

            #Heat Flux
            hf_model=param(ols_fit())[2]
            #Sublimation Heat
            sh_model=param(ols_fit())[4]
            #Mass of Sublimed Ice 
            msi_model=param(ols_fit())[3]
            #Product Dried Layer
            pdl_model=param(ols_fit())[5]
            #Cake Resistance
            cake_res_model=param(ols_fit())[0]
            #Tice_surf
            Tice_model=param(ols_fit())[1]

            #Find the sublimation rate
            def sub_rate(HF):
                sr=np.zeros(HF.size)
                for i in range(len(HF)):
                    if i==0:
                        sr[i]=0
                    else:
                        sr[i]=vial_numbers*HF[i]*np.pi*math.pow(Dout,2)/(4*676)
                return sr
            sr=sub_rate(param(ols_fit())[2]) #g/s
            sr_kg=sr*3600/1000

            #Find the percent removed water
            def perc_wat():
                prw = ((((param(ols_fit()))[3]/water_cont)/fill_volume)/den)*100
                return prw 
            prw=perc_wat()

            #Increment the length of necessary variables by one
            count=count+1

            #Set prw_v as the new value
            prw_v=prw[prw.shape[0]-1] #Pick the last element of the prw array

        #Predict Drying Time Using the final output of the for loop for the prw
        drying_time=cycle_time[cycle_time.shape[0]-1]
        print("Drying Time(Hrs):", drying_time)
        #Predict Product Temperature Based on Maximum Temperature in the Array
        product_temp=np.max(T_product)
        print("Product Temperature(C):",product_temp)
        #Predict Pmin based on parameters set
        p_min_model=p_min_param[0]+p_min_param[1]*sr_kg+p_min_param[2]*sr_kg*sr_kg
        #Store Model Predictions for Drying Time and Product Temperature 
        dt[i,j]=drying_time
        tp[i,j]=product_temp
#         #Store Pmin
#         print(p_min_model.shape)
#         p_min[i,j,:]=p_min_model[0:59]


Current Pressure/Temp Combo: -30.0 C 50.0 mTorr
Current Iteration: 1
Current Iteration: 2
Current Iteration: 3
Current Iteration: 4
Current Iteration: 5
Current Iteration: 6
Current Iteration: 7
Current Iteration: 8
Current Iteration: 9
Current Iteration: 10
Current Iteration: 11
Current Iteration: 12
Current Iteration: 13
Current Iteration: 14
Current Iteration: 15
Current Iteration: 16
Current Iteration: 17
Current Iteration: 18
Current Iteration: 19
Current Iteration: 20
Current Iteration: 21
Current Iteration: 22
Current Iteration: 23
Current Iteration: 24
Current Iteration: 25
Current Iteration: 26
Current Iteration: 27
Current Iteration: 28
Current Iteration: 29
Current Iteration: 30
Current Iteration: 31
Current Iteration: 32
Current Iteration: 33
Current Iteration: 34
Current Iteration: 35
Current Iteration: 36
Current Iteration: 37
Current Iteration: 38
Current Iteration: 39
Current Iteration: 40
Current Iteration: 41
Current Iteration: 42
Current Iteration: 43
Current Iterati

Current Iteration: 42
Current Iteration: 43
Current Iteration: 44
Current Iteration: 45
Current Iteration: 46
Current Iteration: 47
Current Iteration: 48
Current Iteration: 49
Current Iteration: 50
Drying Time(Hrs): 76
Product Temperature(C): -35.1446036156843
(59,)
Current Pressure/Temp Combo: -30.0 C 85.0 mTorr
Current Iteration: 1
Current Iteration: 2
Current Iteration: 3
Current Iteration: 4
Current Iteration: 5
Current Iteration: 6
Current Iteration: 7
Current Iteration: 8
Current Iteration: 9
Current Iteration: 10
Current Iteration: 11
Current Iteration: 12
Current Iteration: 13
Current Iteration: 14
Current Iteration: 15
Current Iteration: 16
Current Iteration: 17
Current Iteration: 18
Current Iteration: 19
Current Iteration: 20
Current Iteration: 21
Current Iteration: 22
Current Iteration: 23
Current Iteration: 24
Current Iteration: 25
Current Iteration: 26
Current Iteration: 27
Current Iteration: 28
Current Iteration: 29
Current Iteration: 30
Current Iteration: 31
Current Iter

Current Iteration: 30
Current Iteration: 31
Current Iteration: 32
Current Iteration: 33
Current Iteration: 34
Current Iteration: 35
Current Iteration: 36
Current Iteration: 37
Current Iteration: 38
Current Iteration: 39
Current Iteration: 40
Current Iteration: 41
Current Iteration: 42
Current Iteration: 43
Current Iteration: 44
Current Iteration: 45
Current Iteration: 46
Current Iteration: 47
Current Iteration: 48
Current Iteration: 49
Current Iteration: 50
Drying Time(Hrs): 76
Product Temperature(C): -34.16423084199906
(59,)
Current Pressure/Temp Combo: -30.0 C 120.0 mTorr
Current Iteration: 1
Current Iteration: 2
Current Iteration: 3
Current Iteration: 4
Current Iteration: 5
Current Iteration: 6
Current Iteration: 7
Current Iteration: 8
Current Iteration: 9
Current Iteration: 10
Current Iteration: 11
Current Iteration: 12
Current Iteration: 13
Current Iteration: 14
Current Iteration: 15
Current Iteration: 16
Current Iteration: 17
Current Iteration: 18
Current Iteration: 19
Current It

Current Iteration: 18
Current Iteration: 19
Current Iteration: 20
Current Iteration: 21
Current Iteration: 22
Current Iteration: 23
Current Iteration: 24
Current Iteration: 25
Current Iteration: 26
Current Iteration: 27
Current Iteration: 28
Current Iteration: 29
Current Iteration: 30
Current Iteration: 31
Current Iteration: 32
Current Iteration: 33
Current Iteration: 34
Current Iteration: 35
Current Iteration: 36
Current Iteration: 37
Current Iteration: 38
Current Iteration: 39
Current Iteration: 40
Current Iteration: 41
Current Iteration: 42
Current Iteration: 43
Current Iteration: 44
Current Iteration: 45
Current Iteration: 46
Current Iteration: 47
Current Iteration: 48
Current Iteration: 49
Current Iteration: 50
Drying Time(Hrs): 76
Product Temperature(C): -33.234756631960884
(59,)
Current Pressure/Temp Combo: -30.0 C 155.0 mTorr
Current Iteration: 1
Current Iteration: 2
Current Iteration: 3
Current Iteration: 4
Current Iteration: 5
Current Iteration: 6
Current Iteration: 7
Current

Current Iteration: 7
Current Iteration: 8
Current Iteration: 9
Current Iteration: 10
Current Iteration: 11
Current Iteration: 12
Current Iteration: 13
Current Iteration: 14
Current Iteration: 15
Current Iteration: 16
Current Iteration: 17
Current Iteration: 18
Current Iteration: 19
Current Iteration: 20
Current Iteration: 21
Current Iteration: 22
Current Iteration: 23
Current Iteration: 24
Current Iteration: 25
Current Iteration: 26
Current Iteration: 27
Current Iteration: 28
Current Iteration: 29
Current Iteration: 30
Current Iteration: 31
Current Iteration: 32
Current Iteration: 33
Current Iteration: 34
Current Iteration: 35
Current Iteration: 36
Current Iteration: 37
Current Iteration: 38
Current Iteration: 39
Current Iteration: 40
Current Iteration: 41
Current Iteration: 42
Current Iteration: 43
Current Iteration: 44
Current Iteration: 45
Current Iteration: 46
Current Iteration: 47
Current Iteration: 48
Current Iteration: 49
Current Iteration: 50
Drying Time(Hrs): 76
Product Temper

KeyboardInterrupt: 

## Save the Output in an Excel Sheet

In [7]:
#Create meshgrid
xx,yy=np.meshgrid(temp,press)

#Save Data as .csv
li = zip(xx.flatten(order='F'), yy.flatten(order='F'), dt.flatten(), tp.flatten())
df_out=pd.DataFrame(data=li,columns=["Shelf Temperature(C)","Chamber Pressure(mTorr)","Drying Time(Hrs)","Product Temperature(C)"])
df_out.to_csv(r'C:\Users\sbadih\OneDrive - Gilead Sciences\Trodelvy 2.0 Modelling Data\Outputs\Design Space\data_log(Edge)_F4.csv')